<a href="https://colab.research.google.com/github/sneha3322/Sentiment-Analysis-Chatbot/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Develop a chatbot equipment with sentiment analysis capabilities.The chatbot will analyze the sentiment of the user's input. The sentiment analysis component will determine whether the user's message expresses a positive,negative, or neutral sentiment. This project combines natural language processing (NLP) techniques, machine learning algorithms to find the sentiment of User.**

In [ ]:
!pip install numpy pandas scikit-learn nltk tensorflow transformers


In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 99% 80.0M/80.9M [00:01<00:00, 44.5MB/s]
100% 80.9M/80.9M [00:01<00:00, 51.0MB/s]


In [ ]:
import zipfile
import pandas as pd

# Unzip the downloaded file
with zipfile.ZipFile("sentiment140.zip", 'r') as zip_ref:
    zip_ref.extractall("sentiment140")

# Load the CSV file into a DataFrame
columns = ['target', 'id', 'date', 'flag', 'user', 'text']
df = pd.read_csv('sentiment140/training.1600000.processed.noemoticon.csv', encoding='latin-1', names=columns)

# Display the first few rows of the dataset
print(df.head())


   target          id                          date      flag  \
0       0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY   
1       0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   
2       0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY   
3       0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
4       0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   

              user                                               text  
0  _TheSpecialOne_  @switchfoot http://twitpic.com/2y1zl - Awww, t...  
1    scotthamilton  is upset that he can't update his Facebook by ...  
2         mattycus  @Kenichan I dived many times for the ball. Man...  
3          ElleCTF    my whole body feels itchy and like its on fire   
4           Karoli  @nationwideclass no, it's not behaving at all....  


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

# Download stopwords and punkt tokenizer
nltk.download('punkt')
nltk.download('stopwords')

# Define stop words
stop_words = set(stopwords.words('english'))

# Preprocess text
def preprocess_text(text):
    # Tokenize the text
    words = word_tokenize(text)
    # Remove non-alphabetic words and convert to lowercase
    words = [word.lower() for word in words if word.isalpha()]
    # Remove stopwords
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

# Apply preprocessing to the 'text' column
df['cleaned_text'] = df['text'].apply(preprocess_text)

# Vectorize the text data
vectorizer = TfidfVectorizer(max_features=3000)
X = vectorizer.fit_transform(df['cleaned_text'])

# Map target to positive (1) and negative (0) labels
y = df['target'].apply(lambda x: 1 if x == 4 else 0)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the model
model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))


Accuracy: 0.7663208333333333
              precision    recall  f1-score   support

           0       0.78      0.74      0.76    239361
           1       0.75      0.79      0.77    240639

    accuracy                           0.77    480000
   macro avg       0.77      0.77      0.77    480000
weighted avg       0.77      0.77      0.77    480000



In [ ]:
def chatbot_response(user_input):
    # Preprocess user input
    processed_input = preprocess_text(user_input)
    # Vectorize the input
    vectorized_input = vectorizer.transform([processed_input])
    # Predict the sentiment probabilities
    sentiment_proba = model.predict_proba(vectorized_input)[0]

    # Set thresholds
    positive_threshold = 0.6
    negative_threshold = 0.4

    if sentiment_proba[1] >= positive_threshold:
        return "I'm glad to hear that!"
    elif sentiment_proba[0] >= positive_threshold:
        return "I'm sorry you're feeling this way. How can I help?"
    else:
        return "It sounds like you're feeling neutral. How can I assist you?"

# Chatbot loop
print("Chatbot: Hello! How are you feeling today? (Type 'exit' to quit)")
while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        break
    response = chatbot_response(user_input)
    print(f"Chatbot: {response}")


Chatbot: Hello! How are you feeling today? (Type 'exit' to quit)
You: im very happy
Chatbot: I'm glad to hear that!
You: but feeling sad as well
Chatbot: I'm sorry you're feeling this way. How can I help?
You: im also confused 
Chatbot: I'm sorry you're feeling this way. How can I help?
You: can i go there are or not 
Chatbot: It sounds like you're feeling neutral. How can I assist you?
You: can i sleep or not now?
Chatbot: It sounds like you're feeling neutral. How can I assist you?
You: ok tqs for responding 
Chatbot: I'm glad to hear that!
You: exit
